In [ ]:
import json
import re
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed
import sys  # sys.exit()를 위해 추가

# --- 설정 ---

# 1. 본인의 KIPRIS Open API 액세스 키로 변경하세요.
ACCESS_KEY = "kAwCm7ZaE6UOsXvimKrzKonFPb9DET5u9vHVzHES8p4="

# 2. KIPRIS API URL
API_BASE_URL = (
    "http://plus.kipris.or.kr/openapi/rest/ClassificationService/searchIPCInfo"
)

# --- 1. CSV에서 s, c, u 맵 로드 ---


def load_csv_map(file_path):
    """CSV 파일에서 IPC 코드와 한글 설명 맵을 로드합니다."""
    try:
        # 1. UTF-8로 먼저 시도
        df = pd.read_csv(file_path, encoding="utf-8")
    except UnicodeDecodeError:
        # 2. 실패 시 'cp949' (euc-kr)로 재시도 (Windows 환경 CSV)
        print("UTF-8 디코딩 실패. 'cp949' (euc-kr)로 다시 시도합니다.")
        try:
            df = pd.read_csv(file_path, encoding="cp949")
        except Exception as e_cp949:
            print(f"CSV 파일 로드 실패 (cp949): {e_cp949}")
            return {}
    except FileNotFoundError:
        print(f"오류: CSV 파일({file_path})을 찾을 수 없습니다.")
        return {}
    except pd.errors.EmptyDataError:
        print(f"오류: CSV 파일({file_path})이 비어있습니다.")
        return {}
    except Exception as e:
        print(f"CSV 파일 로드 중 알 수 없는 오류 발생: {e}")
        return {}

    # 컬럼 이름의 양 끝 공백 제거 (예: " IPC 코드 " -> "IPC 코드")
    df.columns = df.columns.str.strip()

    # 필수 컬럼 확인
    if "IPC 코드" not in df.columns or "IPC 한글설명" not in df.columns:
        print(f"오류: CSV 파일에 'IPC 코드' 또는 'IPC 한글설명' 컬럼이 없습니다.")
        print(f"감지된 컬럼: {df.columns.tolist()}")
        return {}

    # 맵 생성 (IPC 코드의 공백도 제거)
    csv_map = df.set_index(df["IPC 코드"].str.strip())["IPC 한글설명"].to_dict()
    print(f"CSV 맵 로드 완료. {len(csv_map)}개 항목.")
    return csv_map


# --- 2. KIPRIS API에서 세부 항목(m, 1, 2...) 한글 설명 가져오기 ---


def get_korean_title_from_api(ipc_code, access_key):
    """
    KIPRIS API를 호출하여 IPC 코드의 한글 설명을 가져옵니다.
    'A01B1/00' -> 'A01B 1/00' 형식으로 변환하여 요청합니다.
    """

    # API 요청 형식에 맞게 코드 변환 (예: 'A01B1/00' -> 'A01B 1/00')
    formatted_ipc = re.sub(r"([A-Z0-9]{4})([0-9/.]+)$", r"\1 \2", ipc_code)

    params = {"ipcNumber": formatted_ipc, "accessKey": access_key}

    try:
        response = requests.get(API_BASE_URL, params=params, timeout=10)
        response.raise_for_status()  # 200 OK가 아니면 오류 발생
        root = ET.fromstring(response.text)

        result_code = root.find(".//resultCode")
        if (
            result_code is not None
            and result_code.text
            and not str(result_code.text).startswith("00")
        ):
            print(f"API 오류 ({ipc_code}): {root.find('.//resultMsg').text}")
            return None

        kor_desc_element = root.find(".//korDescription")

        if kor_desc_element is not None and kor_desc_element.text:
            return kor_desc_element.text.strip()  # .strip() 추가
        else:
            print(f"경고: {ipc_code}의 <korDescription>을 찾을 수 없음.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류 ({ipc_code}): {e}")
        return None
    except ET.ParseError as e:
        print(f"XML 파싱 오류 ({ipc_code}): {e}")
        return None


# --- 3. 메인 처리 로직 ---


def process_item(item, csv_map, access_key):
    """단일 아이템을 처리하여 title_kr을 추가합니다."""
    kind = item["kind"]
    ipc_code = item["ipc_code"]
    # 원본 'title_en'을 'title_kr'로 바꾸고, 값을 가져옴 (폴백 대비)
    title_en = item.pop("title_en")

    title_kr = None

    if kind in ("s", "c", "u"):
        # CSV 맵에서 조회
        title_kr = csv_map.get(ipc_code.strip())
        if not title_kr:
            print(f"경고: CSV 맵에서 {ipc_code}를 찾을 수 없음.")
    else:
        # API에서 조회 (m, 1, 2 등)
        title_kr = get_korean_title_from_api(ipc_code, access_key)

    # title_kr을 찾지 못한 경우, 원본 영어 설명을 폴백으로 사용
    item["title_kr"] = title_kr if title_kr else title_en
    return item


# --- 실행 ---

if __name__ == "__main__":

    # 0. 액세스 키 확인
    if ACCESS_KEY == "YOUR_KIPRIS_ACCESS_KEY":
        print(
            "오류: 스크립트 상단의 'ACCESS_KEY'를 본인의 KIPRIS 액세스 키로 변경해주세요."
        )
        sys.exit()  # 스크립트 종료

    # 1. 파일 경로 입력 받기
    json_file_path = "../raw/json/ipc_filtered_data_formatted.json"
    csv_file_path = "../raw/ipc_kor.csv"
    output_json_file = "../processed_kor.json"

    # 2. JSON 파일 로드
    try:
        with open(json_file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"{json_file_path}에서 총 {len(data)}개의 항목을 로드했습니다.")
    except FileNotFoundError:
        print(f"오류: JSON 파일({json_file_path})을 찾을 수 없습니다.")
        sys.exit()
    except json.JSONDecodeError:
        print(f"오류: JSON 파일({json_file_path})의 형식이 올바르지 않습니다.")
        sys.exit()
    except Exception as e:
        print(f"JSON 파일 로드 중 오류 발생: {e}")
        sys.exit()

    # 3. CSV 맵 로드
    csv_map = load_csv_map(csv_file_path)

    if not csv_map:
        print("CSV 맵 로드 실패. 스크립트를 종료합니다.")
        sys.exit()

    # 4. 데이터 처리 (병렬 처리)
    print("IPC 데이터 처리 시작 (API 호출 포함)...")
    processed_data = []

    # ThreadPoolExecutor를 사용하여 API 호출을 병렬로 처리
    with ThreadPoolExecutor(max_workers=10) as executor:
        # 각 항목에 대해 process_item 함수를 실행하도록 예약
        futures = {
            executor.submit(process_item, item, csv_map, ACCESS_KEY): item
            for item in data
        }

        for future in as_completed(futures):
            try:
                processed_item = future.result()
                processed_data.append(processed_item)
            except Exception as e:
                # API 호출 대상 항목의 ipc_code를 식별하기 어려워짐 (필요시 future 맵핑 방식 변경)
                print(f"아이템 처리 중 오류 발생: {e}")

    # 5. 원본 순서를 유지하기 위해 'path' 기준으로 정렬
    processed_data.sort(key=lambda x: len(x["path"]))

    # 6. 결과 파일로 저장
    try:
        with open(output_json_file, "w", encoding="utf-8") as f:
            json.dump(processed_data, f, indent=2, ensure_ascii=False)
        print("\n" + "=" * 30)
        print(f"🎉 최종 변환 완료! 결과가 {output_json_file}에 저장되었습니다.")
        print("=" * 30)
    except Exception as e:
        print(f"결과 파일 저장 중 오류 발생: {e}")